In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
%cd /content/drive/MyDrive/justnlp/summarization

/content/drive/MyDrive/justnlp/summarization


# Train Data

In [9]:
import json
import pandas as pd
def load_jsonl(path):
    records = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            records.append(json.loads(line))
    return pd.DataFrame(records)

train_path='./dataset/train/train_judg.jsonl'
judg=load_jsonl(train_path)

In [ ]:
judg.head()

,ID,Judgment
0,id_10,Case :- WRIT - C No. - 11383 of 2023\nPetition...
1,id_1000,Versus\nAppearance:\nand\nDate : 22/09/2023\n1...
2,id_1001,Non-Reportable\nCriminal Appeal No._________ o...
3,id_1002,---- Appellant\nVersus\nUmesh Sharma S/o Late...
4,id_1003,1Whether Reporters of Local Papers may be allo...


In [ ]:
judg.shape

(1200, 2)

In [10]:
import nltk
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [11]:
judg["word_count"] = judg["Judgment"].apply(lambda x: len(nltk.word_tokenize(x)))
judg["sentence_count"] = judg["Judgment"].apply(lambda x: len(nltk.sent_tokenize(x)))
print(judg[["ID", "word_count", "sentence_count"]])

           ID  word_count  sentence_count
0       id_10        3949             258
1     id_1000        1938              51
2     id_1001        1574              56
3     id_1002        2386              79
4     id_1003        2008              60
...       ...         ...             ...
1195   id_995        8844             309
1196   id_996        9094             346
1197   id_997        1135              33
1198   id_998        2194              80
1199   id_999        2854             100

[1200 rows x 3 columns]


In [ ]:
judg['word_count'].min()

169

In [ ]:
judg['word_count'].max()

149745

In [ ]:
judg['word_count'].mean()

np.float64(8368.1275)

In [ ]:
judg['Judgment'].isnull().sum()

np.int64(0)

In [ ]:
print(judg['sentence_count'].min())
print(judg['sentence_count'].max())
print(judg['sentence_count'].mean())

1
5690
284.4691666666667


In [14]:
import re
def clean_text(df, column):
  df[column]=df[column].apply(lambda x: x.lower())
  df[column] = df[column].apply(lambda x:re.sub(r'[^\x00-\x7F]+', ' ', x))
  df[column] = df[column].apply(lambda x: re.sub(r'-\s*\n\s*', '', x))
  df[column] = df[column].apply(lambda x: re.sub(r'\n{2,}', '\n\n', x))
  df[column]=df[column].apply(lambda x: re.sub(r'([^\n])\n([^\n])', r'\1 \2',x))
  df[column] = df[column].apply(lambda x:  re.sub(r'[^\x09\x0A\x0D\x20-\x7E]', ' ', x))
  df[column] = df[column].apply(lambda x: re.sub(r"\s+", " ", x))
  print("after cleaning")
  df["word_count"] = df[column].apply(lambda x: len(nltk.word_tokenize(x)))
  df["sentence_count"] = df[column].apply(lambda x: len(nltk.sent_tokenize(x)))
  print(df[["ID", "word_count","sentence_count"]])
  print(df['word_count'].min())
  print(df['word_count'].max())
  print(df['word_count'].mean())
  return df

In [15]:
judg=clean_text(judg,'Judgment')

after cleaning
           ID  word_count  sentence_count
0       id_10        3496             291
1     id_1000        1856              52
2     id_1001        1465              56
3     id_1002        2297              77
4     id_1003        1899              59
...       ...         ...             ...
1195   id_995        7797             256
1196   id_996        8534             268
1197   id_997        1063              27
1198   id_998        2049              57
1199   id_999        2680              82

[1200 rows x 3 columns]
161
140295
7731.105


In [36]:
from collections import Counter
def extract_sentences(docs):
    all_sents = []
    for doc in docs:
        sentences = nltk.sent_tokenize(doc)
        #f_sent=[s for s in sentences if len(s.split()) > 1]
        all_sents.extend(sentences)
    return all_sents

# Collect sentences from all judgments
all_sents = extract_sentences(judg["Judgment"])

# Count most common sentences
sent_counter = Counter(all_sents)

# Show top 20
print(sent_counter.most_common(100))


[('no.', 866), ('.', 693), ('ii.', 239), ('explanation.', 196), ('iii.', 188), ('iv.', 135), ('nos.', 130), ('case no.', 129), ('154 of 20 20 vinod dua vs. union of india ors.', 117), ('addl.', 111), ('page no.', 105), ('w.p.', 97), ('the authenticity of the order can be re verified from delhi high court order portal by scanning the qr code shown above.', 93), ('crl.', 87), ('cr.', 87), ('c no.', 86), ('07 2019 cnr no.', 82), ('the ld.', 78), ('ld.', 76), ('vi.', 73), ('ors.', 69), ('i.a.', 68), ('vii.', 56), ('no objection to the premature release of the prisoner.', 56), ('emphasis is mine.', 56), ('sh.', 55), ('dg int.', 49), ('act.', 49), ('31 2021 page no.', 47), ('misc.', 46), ('competition appeal at no.', 43), ('o.p.', 41), ('viii.', 41), ('no costs.', 39), ('kunal kamra v union of india connected matters per gs patel j oswpl 9792 2023 j f gspatelj.', 39), ('ix.', 38), ('explanation .', 37), ('thru.', 36), ('respondent no.', 35), ('secy.', 34), ('xi.', 34), ('illustrations.', 34)

In [37]:
sent_counter=sent_counter.most_common(100)
sent_boiler=[s for s in sent_counter if len(s[0].split())<6]

In [38]:
sent_boiler

[('no.', 866),
 ('.', 693),
 ('ii.', 239),
 ('explanation.', 196),
 ('iii.', 188),
 ('iv.', 135),
 ('nos.', 130),
 ('case no.', 129),
 ('addl.', 111),
 ('page no.', 105),
 ('w.p.', 97),
 ('crl.', 87),
 ('cr.', 87),
 ('c no.', 86),
 ('07 2019 cnr no.', 82),
 ('the ld.', 78),
 ('ld.', 76),
 ('vi.', 73),
 ('ors.', 69),
 ('i.a.', 68),
 ('vii.', 56),
 ('emphasis is mine.', 56),
 ('sh.', 55),
 ('dg int.', 49),
 ('act.', 49),
 ('31 2021 page no.', 47),
 ('misc.', 46),
 ('competition appeal at no.', 43),
 ('o.p.', 41),
 ('viii.', 41),
 ('no costs.', 39),
 ('ix.', 38),
 ('explanation .', 37),
 ('thru.', 36),
 ('respondent no.', 35),
 ('secy.', 34),
 ('xi.', 34),
 ('illustrations.', 34),
 ('wp no.', 33),
 ('mtrs.', 30),
 ('supra .', 28),
 ('writ petition criminal no.', 27),
 ('p.c.', 26),
 ('of gujarat.', 26),
 ('smt.', 25),
 ('illustration .', 25),
 ('p .', 25),
 ('vs. u.p.', 25),
 ('sunni central waqf board etc.', 25),
 ('kat .', 24),
 ('of india.', 24),
 ('xii.', 24),
 ('rs.', 24),
 ('25 10 2

In [39]:
from collections import Counter
import re

def tokenize_words(text):
    text = text.lower()
    words = re.findall(r'\b[a-z\.]+\b', text)
    return words

all_tokens = []
for doc in judg["Judgment"]:
    all_tokens.extend(tokenize_words(str(doc)))

word_freq = Counter(all_tokens)
print(word_freq.most_common(50))


[('the', 770535), ('of', 443659), ('to', 242141), ('and', 201579), ('in', 189500), ('a', 133319), ('that', 123290), ('is', 123258), ('be', 84236), ('by', 84036), ('for', 77444), ('on', 74287), ('as', 71354), ('or', 65660), ('it', 60264), ('not', 58112), ('was', 55854), ('court', 54974), ('.', 51547), ('this', 46305), ('with', 46085), ('has', 43005), ('which', 42222), ('under', 37378), ('section', 34709), ('any', 33180), ('an', 33145), ('are', 32581), ('act', 32566), ('no', 29306), ('from', 28710), ('state', 27751), ('have', 26800), ('at', 26388), ('case', 25953), ('been', 25433), ('such', 25316), ('s', 24333), ('he', 23983), ('also', 21148), ('would', 19770), ('order', 19608), ('his', 18663), ('may', 17841), ('other', 17483), ('law', 17299), ('shall', 17248), ('there', 17234), ('had', 17020), ('if', 15906)]


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000, stop_words="english")
X = vectorizer.fit_transform(judg["Judgment"].astype(str))

avg_tfidf = np.asarray(X.mean(axis=0)).ravel()
terms = vectorizer.get_feature_names_out()

# Sort by lowest average tf-idf → most likely boilerplate
low_tfidf_terms = [t for t, score in sorted(zip(terms, avg_tfidf), key=lambda x: x[1])[:50]]
print("\n🔹 Candidate boilerplate terms (low TF-IDF):")
print(low_tfidf_terms)



🔹 Candidate boilerplate terms (low TF-IDF):
['chakravarthi', 'naroda', 'maharaja', 'fron', '1e', 'accession', 'abstraction', 'recharge', 'ofhis', 'ebix', 'nathuram', 'karkare', 'godse', 'dominion', '124f', 'lgbtq', 'sreekumar', 'informational', 'hin', 'rpa', 'fma', 'spv', 'axykno', 'wheat', '051', 'invalidity', 'nundy', 'thot', 'atthe', 'comparative', 'importer', '342a', 'gulberg', 'badge', 'bythe', 'apte', 'irrevocable', 'philip', 'chronicle', 'shipping', 'stakes', '13a', 'oems', 'dargah', 'ond', 'estates', 'pooling', 'signatories', 'jv', 'deccan']


In [42]:
boilerplate_dict = {
    "sentences": [s for s, _ in sent_boiler],
    "terms": ['fron', 'ebix', 'hin', 'rpa', 'fma', 'spv', 'axykno', 'thot', 'apte','ond', 'jv', 'deccan']
}

import json
with open("boilerplate_candidates.json", "w", encoding="utf-8") as f:
    json.dump(boilerplate_dict, f, indent=2, ensure_ascii=False)

print("\n✅ Boilerplate candidates saved to boilerplate_candidates.json")


✅ Boilerplate candidates saved to boilerplate_candidates.json


In [ ]:
train_ref_path='./dataset/train/train_ref_summ.jsonl'
ref=load_jsonl(train_ref_path)

In [ ]:
ref.head()

,ID,Summary
0,id_10,The Allahabad High Court on Thursday stayed th...
1,id_1000,A convict in Gujarat who had secured bail in 2...
2,id_1001,A police officer failing in their fundamental ...
3,id_1002,The Chhattisgarh High Court recently observed ...
4,id_1003,The Gujarat High Court recently quashed a Firs...


In [ ]:
ref.shape

(1200, 2)

In [ ]:
ref["word_count"] = ref["Summary"].apply(lambda x: len(nltk.word_tokenize(x)))
ref["sentence_count"] = ref["Summary"].apply(lambda x: len(nltk.sent_tokenize(x)))

print(ref[["ID", "word_count", "sentence_count"]])

           ID  word_count  sentence_count
0       id_10         759              22
1     id_1000         595              17
2     id_1001         557              17
3     id_1002         444              17
4     id_1003         486              16
...       ...         ...             ...
1195   id_995         775              26
1196   id_996         790              20
1197   id_997         580              18
1198   id_998         463              14
1199   id_999         679              23

[1200 rows x 3 columns]


In [ ]:
print(ref['word_count'].min())
print(ref['word_count'].max())
print(ref['word_count'].mean())

29
2412
609.9591666666666


In [ ]:
print(ref['sentence_count'].min())
print(ref['sentence_count'].max())
print(ref['sentence_count'].mean())

1
73
19.948333333333334


In [ ]:
ref['Summary'].isnull().sum()

np.int64(0)

In [ ]:
nltk.sent_tokenize(ref['Summary'].iloc[10])

['The Supreme Court on Monday granted bail to Sadanand Kadam, an aide of Shiv Sena (UBT) MLC Anil Parab in the Dapoli resort money laundering case.',
 "[Sadanand Gangaram Kadam v Directorate of Enforcement and Anr]\nA bench of Justices Abhay S Oka and Ujjal Bhuyan noted Additional Solicitor General (ASG) Suryaprakash V Raju's fair stand of leaving it to the Court to decide the prayer for bail.",
 'Even otherwise, the Court found that Kadam was entitled to bail in accordance with Section 45(1)(ii) of the Prevention of Money Laundering Act (PMLA) on appropriate terms and conditions, till the disposal of the complaint case filed by the Enforcement Directorate.',
 "In view of the ASG's stand, the Court did not record reasons in its order and directed that Kadam be produced before the Special Court within a week for being released on bail on appropriate terms and conditions.",
 'In March 2022, the income tax (IT) department had raided premises in several districts of Maharashtra linked to a

In [ ]:
judg

,ID,Judgment,word_count,sentence_count
0,id_10,Case WRIT C No. 11383 of 2023 Petitioner Syed ...,3376,263
1,id_1000,Versus Appearance and Date 22092023 1.The pres...,1829,52
2,id_1001,NonReportable Criminal Appeal No. of 2024 Spec...,1452,59
3,id_1002,Appellant Versus Umesh Sharma So Late Ompraka...,2256,79
4,id_1003,1Whether Reporters of Local Papers may be allo...,1876,62
...,...,...,...,...
1195,id_995,arising out of Special Leave Petition Criminal...,7794,309
1196,id_996,Reserved on 15.03.2023 Pronounced on 16.05.20...,8537,350
1197,id_997,Reportable Writ Petition C No 961 of 2021 Neil...,1066,33
1198,id_998,Ms. Biovet Private Limited .Applicants In the ...,1903,74


In [ ]:
ref

,ID,Summary
0,id_10,The Allahabad High Court on Thursday stayed th...
1,id_1000,A convict in Gujarat who had secured bail in 2...
2,id_1001,A police officer failing in their fundamental ...
3,id_1002,The Chhattisgarh High Court recently observed ...
4,id_1003,The Gujarat High Court recently quashed a Firs...
...,...,...
1195,id_995,Facts sourced from a statement made by accused...
1196,id_996,The Delhi High Court recently directed mediati...
1197,id_997,The Supreme Court on Friday passed an interim ...
1198,id_998,The Bombay High Court on Thursday permitted Bi...


In [ ]:
merged_df = judg.merge(ref, on="ID", how="inner")

In [ ]:
merged_df = merged_df[['ID','Judgment','Summary']]
merged_df

,ID,Judgment,Summary
0,id_10,Case WRIT C No. 11383 of 2023 Petitioner Syed ...,The Allahabad High Court on Thursday stayed th...
1,id_1000,Versus Appearance and Date 22092023 1.The pres...,A convict in Gujarat who had secured bail in 2...
2,id_1001,NonReportable Criminal Appeal No. of 2024 Spec...,A police officer failing in their fundamental ...
3,id_1002,Appellant Versus Umesh Sharma So Late Ompraka...,The Chhattisgarh High Court recently observed ...
4,id_1003,1Whether Reporters of Local Papers may be allo...,The Gujarat High Court recently quashed a Firs...
...,...,...,...
1195,id_995,arising out of Special Leave Petition Criminal...,Facts sourced from a statement made by accused...
1196,id_996,Reserved on 15.03.2023 Pronounced on 16.05.20...,The Delhi High Court recently directed mediati...
1197,id_997,Reportable Writ Petition C No 961 of 2021 Neil...,The Supreme Court on Friday passed an interim ...
1198,id_998,Ms. Biovet Private Limited .Applicants In the ...,The Bombay High Court on Thursday permitted Bi...


In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(merged_df, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
import json

def save_jsonl(df, path):
    with open(path, "w", encoding="utf-8") as f:
        for _, row in df.iterrows():
            f.write(json.dumps(row.to_dict(), ensure_ascii=False) + "\n")

# Save to JSONL
save_jsonl(train_df, "./dataset/train/train_split.jsonl")
save_jsonl(val_df, "./dataset/train/val_split.jsonl")

print("Train/Test JSONL files saved!")


Train/Test JSONL files saved!


In [ ]:
print(train_df.shape)
print(val_df.shape)

(960, 3)
(240, 3)


In [ ]:
train_df

,ID,Judgment,Summary
331,id_1498,and W.M.P.MDNo.21615 of 2022 S.Lawrence Vimalr...,Designation of a lawyer as a Senior Advocate i...
409,id_16,This file relates to reconsideration of the pr...,The Union Law Minister told the Supreme Court ...
76,id_1106,1PA.RGMisc.2021 Date24.06.202L Considering the...,The Rajasthan High Court will start regular he...
868,id_539,IN THE HIGH COURT OF MADHYA PRADESHAT JABALPUR...,The Madhya Pradesh High Court recently quashed...
138,id_1205,Crl. A. No.12002023 Criminal Appeal No.1200 of...,While society should be protected from drug-re...
...,...,...,...
1044,id_797,Haji Abdul Gani Khan Anr. Petitioners v. Union...,The Supreme Court on Monday dismissed the peti...
1095,id_863,Khalil Abbas Fakir .. Applicant Tabbasum Khali...,The Bombay High Court recently held that a div...
1130,id_911,Criminal Revisional Jurisdiction Appellate Sid...,The Calcutta High Court recently quashed a che...
860,id_527,1 CC.No.307512021 Dated this the 18th day of O...,An Special Court in Bengaluru recently convict...


In [ ]:
val_df

,ID,Judgment,Summary
1178,id_972,"Through: Mr. Hemant Daswani, Ms.\nSauyma Bajpa...",The Delhi High Court has upheld a single-judge...
865,id_533,ORDER : (Per the Hon’ble the Chief Justice Alo...,The Telangana High Court recently upheld the C...
101,id_1145,"Vasundhara d/o Praful Bhojane\nUnion of India,...",The Bombay High Court today issued notice in a...
439,id_1640,"Reportable\nHigh Court Bar Association, Allaha...",A Constitution bench of the Supreme Court on T...
58,id_1079,Applicant :- Phoolchandra Yadav And 2 Others\n...,The Allahabad High Court has granted bail to t...
...,...,...,...
382,id_1568,Ms. Kangana Ranaut ...Petitioner\nVersus\n1. T...,The Bombay High Court on Friday rejected a pet...
867,id_538,Versus\nAppearance:\nMS URVASHI K MEHTA(11469)...,A rape is a rape even if it is committed by th...
542,id_1791,Reportable\nCivil Appeal No. 8129 of 2022\nMad...,The Central government's refusal to renew broa...
1193,id_993,Writ Petition Nos.16650 of 2020\nand 144 48 of...,The Madras High Court on Monday upheld the con...


In [ ]:
val_path='./dataset/validation/val_judg.jsonl'
val=load_jsonl(train_path)

In [ ]:
nltk.sent_tokenize(val['Judgment'].iloc[1])

['Versus\nAppearance:\nand\nDate : 22/09/2023\n1.The present case is an eye opener.',
 'The convict-\nChandanji @ Gato Chhanaji Thakor has filed the\npresent application seeking regular bail through\njail.',
 'Such application was filed by him on\n05.08.2023, which is forwarded to the Registry of\nthis Court vide communication dated 11.08.2023\nwritten by the Deputy Superintendent of Ahmedabad\nCentral Jail.',
 '2.When the matter was listed yesterday, learned\nadvocate Mr.Soni appearing for the applicant-convict\nhas invited attention of this Court to the order\ndated 29.09.2020 passed in Criminal Misc.',
 'Application\n(for suspension of sentence) No.1 of 2020 in the\ncaptioned appeal and has submitted that this Court,\nafter passing a comprehensive order, had already\nreleased the applicant on regular bail by suspending\nhis sentence under the provision of Section 389 of\nthe Code of Criminal Procedure, 1973 (for short "the\n3.The matter was ordered to be listed today and\nlearned AP

In [ ]:

val["word_count"] = val["Judgment"].apply(lambda x: len(nltk.word_tokenize(x)))
val["sentence_count"] = val["Judgment"].apply(lambda x: len(nltk.sent_tokenize(x)))

print(val[["ID", "word_count", "sentence_count"]])

           ID  word_count  sentence_count
0       id_10        3949             258
1     id_1000        1938              51
2     id_1001        1574              56
3     id_1002        2386              79
4     id_1003        2008              60
...       ...         ...             ...
1195   id_995        8844             309
1196   id_996        9094             346
1197   id_997        1135              33
1198   id_998        2194              80
1199   id_999        2854             100

[1200 rows x 3 columns]


In [ ]:
val['Judgment'].isnull().sum()

np.int64(0)

In [ ]:
val=clean_text(val,'Judgment')

after cleaning
           ID  word_count  sentence_count
0       id_10        3376             263
1     id_1000        1829              52
2     id_1001        1452              59
3     id_1002        2256              79
4     id_1003        1876              62
...       ...         ...             ...
1195   id_995        7794             309
1196   id_996        8537             350
1197   id_997        1066              33
1198   id_998        1903              74
1199   id_999        2599             103

[1200 rows x 3 columns]
161
138822
7644.554166666667


In [ ]:
val = val[['ID','Judgment']]
save_jsonl(val_df, "./dataset/validation/val_judgclean.jsonl")

In [ ]:
val

,ID,Judgment
0,id_10,Case WRIT C No. 11383 of 2023 Petitioner Syed ...
1,id_1000,Versus Appearance and Date 22092023 1.The pres...
2,id_1001,NonReportable Criminal Appeal No. of 2024 Spec...
3,id_1002,Appellant Versus Umesh Sharma So Late Ompraka...
4,id_1003,1Whether Reporters of Local Papers may be allo...
...,...,...
1195,id_995,arising out of Special Leave Petition Criminal...
1196,id_996,Reserved on 15.03.2023 Pronounced on 16.05.20...
1197,id_997,Reportable Writ Petition C No 961 of 2021 Neil...
1198,id_998,Ms. Biovet Private Limited .Applicants In the ...


In [ ]:
def split_to_sentences_summ(para):
    sents = tokenize.sent_tokenize(para)
    return sents

def get_chunks_data_from_docV2(doc, summ):
    doc_sents = split_to_sentences_summ(doc)
    summ_sents = split_to_sentences_summ(summ)
    return len(max(doc_sents)), len(max(summ_sents))
mlj=0
mls=0
for _, row in tqdm(df.iterrows(), total=len(df)):
         doc_id = str(row["ID"])
         judgment = row["Judgment"]
         reference = row["Summary"]
         dl, sl = get_chunks_data_from_docV2(judgment,reference)
         print(doc_id, dl)
        #  if dl>mlj:
        #    mlj=dl
        #  if sl>mls:
        #    mls=sl


  2%|▏         | 16/960 [00:00<00:06, 156.69it/s]

id_1498 154
id_16 134
id_1106 42
id_539 4
id_1205 110
id_1792 43
id_273 95
id_610 140
id_1084 181
id_1211 121
id_338 133
id_1559 432
id_1544 110
id_1206 306
id_558 36
id_791 225
id_1334 146
id_175 83
id_14 86
id_961 152
id_1709 131
id_1531 123
id_736 175
id_190 43
id_262 98
id_41 105


  5%|▍         | 44/960 [00:00<00:09, 97.82it/s] 

id_1807 22
id_1835 35
id_600 277
id_1681 106
id_1852 156
id_40 369
id_216 158
id_499 15
id_1430 146
id_105 128
id_1039 140
id_1358 165
id_1629 147
id_1202 331
id_871 231
id_532 4
id_1002 134
id_491 37


  6%|▌         | 57/960 [00:00<00:08, 105.53it/s]

...

id_1714 100
id_1562 147
id_1271 157
id_398 582
id_132 86
id_438 37
id_840 15
id_55 131
id_1508 173
id_343 6
id_1439 130
id_754 100
id_1663 230
id_467 175
id_1406 101
id_722 123
id_1298 191
id_1236 245
id_353 239
id_1589 215
id_316 89
id_91 140
id_1822 3
id_759 53
id_763 117
id_878 56
id_1306 20
id_1044 182
id_410 259
id_1738 169
id_982 53
id_1134 350
id_579 11
id_1821 171


 98%|█████████▊| 937/960 [00:07<00:00, 133.81it/s]

id_412 76
id_1357 127
id_1013 102
id_1471 15
id_1865 68
id_1574 341
id_1244 159
id_502 225
id_1025 54
id_233 54
id_94 138
id_49 73
id_182 90
id_962 21
id_294 123
id_67 522
id_1273 242
id_693 189
id_310 20
id_1743 377
id_848 78
id_1694 84
id_520 573
id_380 7
id_1376 142
id_1026 3
id_1668 68
id_1413 18
id_668 183
id_1572 126
id_454 271
id_1515 165


100%|██████████| 960/960 [00:07<00:00, 132.61it/s]

id_404 315
id_1190 11
id_542 75
id_904 62
id_1119 113
id_1497 113
id_1682 57
id_1175 135
id_797 126
id_863 81
id_911 31
id_527 99
id_907 199
